In [2]:
from helper.neural_net import *
from models.autoencoder import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
run_experiment('autoencoder', 'mse', 'adam', image_dir, test_dir, num_epochs=1, autoencoder=True)

Things will go much quicker with a GPU
Starting training
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder
decoder
torch.Size([100, 16])
torch.Size([100, 128])
torch.Size([100, 16, 4, 4])
torch.Size([100, 3, 16, 16])
decoder

KeyboardInterrupt: 